In [5]:
!git clone https://github.com/ahnobari/MOOCP.git
!pip install -U pymoo svgpath2mpl
!mv ./MOOCP/* ./
!rm -r ./MOOCP

fatal: destination path 'MOOCP' already exists and is not an empty directory.


'mv' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


<font color="red" size="3"><b>WARNING: </b></font> If you plan to work on this notebook in colab we would like to warn you that if you do not download any files you save or save the changes separately, upon the runtime being killed either by you closing the tab/browser or by timing out due to inactivity you WILL LOSE YOUR FILES AND WORK. It is highly recommeneded that you make a local python environment to work.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pymoo

# pymoo sub sections
# from pymoo.core.problem import Problem
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.variable import Real, Integer, Choice, Binary
from pymoo.core.mixed import MixedVariableMating, MixedVariableGA, MixedVariableSampling, MixedVariableDuplicateElimination
from pymoo.optimize import minimize

# Utiliies for working with linkages (DeCoDE Lab)
from linkage_utils import *

# Other modules
import json
from IPython.display import HTML
import matplotlib.animation as animation
from tqdm.autonotebook import trange
import pickle

#set random seed for repeatability
#np.random.seed(5)

In [10]:
def add_population_csv(file_name, population):
    """Save a population of mechanims as csv for submission/evaluation
    ----------
    file_name: str
                File name and path to save the population in.
    population: list [N]
                A list of 1D mechanims representations to be saved in the CSV.
    """
    
    with open(file_name, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', lineterminator = '\n')
        for m in population:
            m = np.array(m)
            writer.writerow(m.tolist())

In [7]:
target_curves = []

# Read every file separately and append to the list
for i in range(20):
    target_curves.append(np.loadtxt('./data/%i.csv'%(i),delimiter=','))

### Submission Format
For your submissions we require a population of mechanisms (will be discussed later) for each target curve. For this purpose you must obtain 1D representations of each member of the population (for each target curve). To make things easier we have provided the <code>to_final_representation(C,x0,fixed_nodes,motor,target)</code> function to you in the <code>linkage_utils module</code>. We also have provided the <code>from_1D_representation(mechanism)</code> function which returns <code>C,x0,fixed_nodes,motor,target</code> from the 1D representation, essentially reversing the process. The code for this is presented in the appendix. 

You will create a .csv file with all the mechanisms each in one row of the .csv file, up to 1000, which is the limit we restrict you to. You will submit a .csv file for each target curve, therefore you will have a total of 20 .csv files. We also ask that you submit the 20 .csv files as a zip file. The naming scheme for the submission will be #.csv where you replace # with the number of the target curve starting from 0.csv and up to 19.csv.

Your performance on the targets will be measured based on the hypervolume calculated from the population. Note that there is a limit on the number of mechanisms you are permitted to submit as potential solutions for any given curve (1000). The closer you get to ideal the higher the score you will receive. The total score for your submission will be the average hypervolume for all 20 target curves:

To make this part easier we have also provided a population to .csv function which will help you make the conversion from python array to .csv for all members of a population. Since we haven't actually generated any mechanisms yet, lets generate some random ones as a placeholder. First let's use the <code>random_generator_ns</code> function we have provided to you to generate 6 random mechanisms (Note: if you pass an <code>n=#</code> parameter to this function and set # to be any number the random mechanism will have that number of nodes). We will add these 6 mechanisms to our example population. 

In [8]:
class mechanism_synthesis_optimization(ElementwiseProblem):
    
    # When intializing, set the mechanism size and target curve
    def __init__(self, target_point_cloud, N = 15):
        self.N = N
        variables = dict()
        
        for i in range(N):
            if i not in range(4):
                variables["C"+str(i)+"_1st"]=Integer(bounds=(0,i-1))
                variables["C"+str(i)+"_2nd"]=Integer(bounds=(0,i-1))  


        # The upper triangular portion of our NxN Connectivity Matrix consists of Nx(N-1)/2 boolean variables:
        #for i in range(N):
        #    for j in range(i):
        #        variables["C" + str(j) + "_" + str(i)] = Binary()
        
        # We Delete C0_1 since we know node 1 is connected to the motor
        #del variables["C0_1"]
        #del variables["C0_1st"]
        #del variables["C0_2nd"]
        #del variables["C1_1st"]
        #del variables["C1_2nd"]
        #del variables["C2_1st"]
        #del variables["C2_2nd"]
        #del variables["C3_1st"]
        #del variables["C3_2nd"]
     
        #print(variables)
                
        #Our position matrix consists of Nx2 real numbers (cartesian coordinate values) between 0 and 1
        for i in range(2*N):
            variables["X0" + str(i)] = Real(bounds=(0.0, 1.0))

        # Our node type vector consists of N boolean variables (fixed vs non-fixed)
        
        #for i in range(2*N):
        #    variables["fixed_nodes" + str(i)] =  Binary(N)

        #del variables["fixed_nodes0"]
        #del variables["fixed_nodes3"]

        # Our target node is an integer between 1 and N-1, (any except the motor node). 
        #variables["target"] = Integer(bounds=(1,N-1))
        #variables["target"] = Integer(N-1)
        
        # Set up some variables in the problem class we inherit for pymoo
        # n_obj=number of objectives, n_constr=number of constraints
        # Our objectives are chamfer distance and material, and they both have constraints.
        super().__init__(vars=variables, n_obj=2, n_constr=2)
        
        # Store the target curve point cloud
        self.tpc = target_point_cloud
        
        # Make a solver instance
        self.solver = mechanism_solver()
        
        # Make a normalizer instance
        self.normalizer = curve_normalizer(scale=False)
    
    def convert_1D_to_mech(self, x):
        N = self.N
        
        # Get motor, target, and driven node values
        #target = x["target"]
        target = N-1
        
        # Build connectivity matrix from its flattened constitutive variables
        C = np.zeros((N,N))
        #x["C0_1"] = 1
        x["C0_1st"] = 1 #connected to the first - not a binary value of 1
        x["C0_2nd"] = 1 #
        x["C1_1st"]=0
        x["C1_2nd"]=2
        x["C2_1st"]=1
        x["C2_2nd"]=3
        x["C3_1st"]=2
        x["C3_2nd"]=2
        
        #for i in range(N):
        #    for j in range(i):
        #        C[i,j] = x["C" + str(j) + "_" + str(i)]
        #        C[j,i] = x["C" + str(j) + "_" + str(i)]

       
        for i in range(N):
            for j in range(i):
                if(x["C"+str(i)+"_1st"]==j or x["C"+str(i)+"_2nd"]==j):
                    C[i,j]=1
                    C[j,i]=C[i,j]
        
        #add representation of C[0,1], C[1,0]
        C[0,1]=1
        C[1,0]=1

        #4 bar linkage embryo
        C[2,1]=1
        C[1,2]=1
        C[3,2]=1
        C[2,3]=1


        #print("the connectivity matrix is:", C)

        x["fixed_nodes0"]=1
        x["fixed_nodes3"]=1
        # Reshape flattened position matrix to its proper Nx2 shape 
        x0 = np.array([x["X0" + str(i)] for i in range(2*N)]).reshape([N,2])
        
        # Extract a list of Nodes that are fixed from boolean fixed_nodes vector
        #fixed_nodes = np.where(np.array([x["fixed_nodes" + str(i)] for i in range(N)]))[0].astype(np.int)
        fixed_nodes=[0, 3]
        
        #We fix the motor and original ground node as 0 and 1 respectively in this implementation
        motor=np.array([1,0])
        return target, C, x0, fixed_nodes, motor
    
    def fix_mechanism(self, target, C, x0, fixed_nodes, motor):
        
        # Make sure driven node is not grounded
        #fixed_nodes = fixed_nodes[fixed_nodes!=1]
        fixed_nodes=[0, 3]
        motor=np.array([1,0])
        N = np.shape(C)[0]
        #print("target: " + str(target))
        target = N-1
        #print("new target: " + str(target))
        # Make sure target node is not grounded
        #fixed_nodes = fixed_nodes[fixed_nodes!=target]
            
        #There are many other things you can do to correct mechanisms and help your algorithm

        #remove C values that are dependent on target node
        #create a unidirected graph from the base
    
        #print("C is:", C, "target is:", target)
        if target>3 and target<N-1:
          C=C[:target+1, :target+1]
          x0=x0[:target+1,:]
        #else:
          #C=C[:4, :4]
          #x0=x0[:4,:]

        return target, C, x0, fixed_nodes, motor
    
        
    def _evaluate(self, x, out, *args, **kwargs):
        #Convert to mechanism representation
        target, C, x0, fixed_nodes, motor = self.convert_1D_to_mech(x)
        
        #Correct some potential issues in our mechanism to assist our algorithm
        target, C, x0, fixed_nodes, motor = self.fix_mechanism(target, C, x0, fixed_nodes, motor)


        #print(target, C, x0, fixed_nodes, motor)
        # Step 2: Simulate
        x_sol,locking,over_under_defined = self.solver.solve_rev(200,x0,C,motor,fixed_nodes,False)

        # check to see if the mechanism is valid
        if locking or over_under_defined:
            # if mechanism is invalid set the objective to infinity
            out["F"] = [np.Inf,np.Inf]
            out["G"] = [np.Inf,np.inf]
#             out["G"] = [locking,over_under_defined]
        else:
            # Step 3: Normalize
            x_norm = self.normalizer.get_oriented(x_sol[:,target,:])

            # Step 4: Rasterize
            out_pc = rasterized_curve_coords(x_norm,500)
            
            # Step 5: Compare
            cd = chamfer_distance(out_pc,self.tpc)

            # Send the chamfer distance to pymoo as objective
            m = self.solver.material(x0,C)
            out["F"] = [cd,m]

            # Set constraints as CD<=30 and Material<=6
            # Be careful about modifying these - designs that 
            # violate the problem constraints will not be scored.
            out["G"] = [cd - 30, m - 6]
#             out["G"] = [cd - 70, m - 6]



### Now Let's Get a Submission Going

Now lets run the process for all the target curves in a loop:

In [15]:
import time
start_time = time.time()

for i,target_curve in enumerate(target_curves):
    for n in range(5,11):
        print("Current n: " + str(n))
        if n in range(5,8):
            pop_size = 150
            gen_num = 150
        elif n in range(8,12):
            pop_size = 100
            gen_num = 300
        elif n in range(12,21):
            pop_size = 50
            gen_num = 1000
        # Setup Problem
        problem = mechanism_synthesis_optimization(target_curve,n)

        # Get Algorithm
        algorithm = NSGA2(pop_size=pop_size, sampling=MixedVariableSampling(),
                        mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
                        eliminate_duplicates=MixedVariableDuplicateElimination())

        # Run for 150 generations
        results = minimize(problem,
                        algorithm,
                        ('n_gen', gen_num),
                        verbose=True,
                        save_history=True,
                        seed=0,
                        display=best())
        
        count=0
        while results.X is None and count!=2:
            count+=1
            print('Did Not Find Solutions for Target ' + str(i) + '. Trying Seed ' + str(count+1) + '!')
            # Run for 150 generations
            results = minimize(problem,
                            algorithm,
                            ('n_gen', 150),
                            verbose=True,
                            save_history=True,
                            seed=0+count,
                            display=best())
        
        if results.X is None:
            print('Did Not Find Solutions for any Seed!!')
        else:
            mechanisms = []
            for x in results.X:
                target, C, x0, fixed_nodes, motor  = problem.convert_1D_to_mech(x)
                target, C, x0, fixed_nodes, motor  = problem.fix_mechanism(target, C, x0, fixed_nodes, motor )
                mechanisms.append(to_final_representation(C,x0,fixed_nodes,motor,target))
            
            #plt.figure()
            #plt.scatter(results.F[:,1],results.F[:,0])
            #plt.scatter([6],[30],color="red")
            #plt.xlabel('Material Use')
            #plt.ylabel('Chamfer Distance')
            #plt.title('Best Pareto Fronts For Target %i'%(i+1))
            #plt.xlim([0,8.0])
            #plt.ylim([0,40.0])
            #sorted_performance = results.F[np.argsort(results.F[:,1])]
            #sorted_performance = np.concatenate([sorted_performance,[[np.min(results.F[:,0]),6],[30,6],[30,np.min(results.F[:,1])]]])
            #plt.fill(sorted_performance[:,1],sorted_performance[:,0],color="#008cff",alpha=0.2)

            print('Hyper Volume For Target %i = %f' % ((i),hyper_volume(results.F,[30.0,6.0])))
            
            add_population_csv('./results/final/%i.csv'%i,mechanisms)
print("--- Execution time: %s seconds ---" % (time.time() - start_time))

Current n: 5
Hyper Volume For Target 0 = 118.352706
Current n: 6
Hyper Volume For Target 0 = 101.197259
Current n: 7
Hyper Volume For Target 0 = 70.702695
Current n: 8
Hyper Volume For Target 0 = 79.655518
Current n: 9
Hyper Volume For Target 0 = 71.023704
Current n: 10
Hyper Volume For Target 0 = 55.069481
Current n: 5
Hyper Volume For Target 1 = 128.980652
Current n: 6
Hyper Volume For Target 1 = 127.439589
Current n: 7
Hyper Volume For Target 1 = 98.755710
Current n: 8
Hyper Volume For Target 1 = 79.490029
Current n: 9
Hyper Volume For Target 1 = 84.276550
Current n: 10
Hyper Volume For Target 1 = 92.563244
Current n: 5
Hyper Volume For Target 2 = 110.315371
Current n: 6
Hyper Volume For Target 2 = 111.487163
Current n: 7
Hyper Volume For Target 2 = 63.765836
Current n: 8
Hyper Volume For Target 2 = 88.498396
Current n: 9
Hyper Volume For Target 2 = 70.776978
Current n: 10
Hyper Volume For Target 2 = 84.644530
Current n: 5
Hyper Volume For Target 3 = 137.492664
Current n: 6
Hyper Vo

Now Let's see how we did:

In [17]:
evaluate_submission()

100%|██████████| 20/20 [00:52<00:00,  2.61s/it]

Score Break Down:
Curve 0: 121.837560
Curve 1: 135.209466
Curve 2: 121.583202
Curve 3: 143.231011
Curve 4: 132.395915
Curve 5: 107.309728
Curve 6: 113.206574
Curve 7: 145.285340
Curve 8: 153.920429
Curve 9: 106.181119
Curve 10: 128.519867
Curve 11: 118.398509
Curve 12: 157.595425
Curve 13: 143.824375
Curve 14: 150.592294
Curve 15: 75.829190
Curve 16: 153.927256
Curve 17: 117.578373
Curve 18: 134.746573
Curve 19: 145.260543
Overall Score: 130.321637


130.3216374169796

### Extra Material and Challenges With the Current Implemenation

#### Some Important Notes:

<ul>
    <li><b>Random Seeds: </b>If you plan to run things in the notebook please do not forget to remove the random seeds from the code we set the seeds so that the results of the code do not change when you run them again, and you would obviously have better results if you randomize differently every time.</li>
    <li><b>Colab vs Local: </b>If you do plan to use Google Colab for your work remember to download any files you save and the changes to the notebook because every time Google Colab ends the runtime (or it times out) it deletes all files and progress. So <font color="red"><b>BE CAREFUL!</b></font></li> It is generally recommended that you work locally.
</ul>

#### Problems With The Current Implementation:

The current implementation has a few problems (your task is to improve upon this). The first is that the method cannot handle very large sizes of mechanisms and it struggles to find mechanisms that are not locking or under/over defined. Lets see the algorithm try to make mechanisms of size 12. To give it a chance lets run it for 1000 generations:

As you can see it can't get out of the infeasable region. What can we do to fix this problem for larger mechanisms??

Since we know that our mechanism has one degree of freedom, does the current way we have represented our mechanism really the most efficient way to do this? Does the current Mutation/Crossover/Initialization make sense?  Is Genetic Algorthim really the best way to do this? What are the ways you can improve this method?

Understanding evolutionary design of linkages is key to developing better methods for this process. Look at :


<font size="3"><b>Lipson, H. (2008). Evolutionary synthesis of kinematic mechanisms. Artificial Intelligence for Engineering Design, Analysis and Manufacturing, 22(3), 195-205. </b></font> [Online PDF](https://www.cambridge.org/core/journals/ai-edam/article/evolutionary-synthesis-of-kinematic-mechanisms/6DF594784096ECD1C66E9F8CB9AAB0AE)

<font size="3"><b>Bacher, M., Coros, S., Thomaszewski, B. (2015). LinkEdit: interactive linkage editing using symbolic kinematics. ACM Transactions on Graphics (TOG), 34(4), 99.</b></font> [Online (Access Through MIT)](https://libproxy.mit.edu/login?url=https://dl.acm.org/doi/10.1145/2766985)

Other references:

<font size="2">Deshpande, Shrinath, en Anurag Purwar. A Machine Learning Approach to Kinematic Synthesis of Defect-Free Planar Four-Bar Linkages. Vol 5B: 42nd Mechanisms and Robotics Conference. International Design Engineering Technical Conferences and Computers and Information in Engineering Conference, 08 2018.</font> [Online (Access Through MIT)](https://libproxy.mit.edu/login?url=https://doi.org/10.1115/DETC2018-85578)


<font size="2">Vermeer, Kaz, Reinier Kuppens, en Justus Herder. Kinematic Synthesis Using Reinforcement Learning. Vol 2A: 44th Design Automation Conference. International Design Engineering Technical Conferences and Computers and Information in Engineering Conference, 08 2018.</font> [Online (Access Through MIT)](https://libproxy.mit.edu/login?url=https://doi.org/10.1115/DETC2018-85529)